In [1]:
import pandas as pd
import numpy as np

import scipy.stats as st

from data_cleaning import df_webfullfilter, pivot_table, test_group, control_group,t_stat, p_value, alpha, control_total_completion_rate, test_total_completion_rate, map_categoria

from statsmodels.stats.proportion import proportions_ztest

c:\Users\Lymaris\ih-labs\lab-vanguard-ab-test\data_cleaning.py:70: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_webfull['date_time'] = pd.to_datetime(df_webfull['date_time'], dayfirst=True)
c:\Users\Lymaris\ih-labs\lab-vanguard-ab-test\data_cleaning.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_webfullfilter['experiment'] = df_webfullfilter['client_id'].map(map_categoria)


### Hypotese 1


#### Completion Rate 

In [2]:
print(f"Control Group Completion Rate: {control_total_completion_rate * 100:.2f}%")
print(f"Test Group Completion Rate: {test_total_completion_rate * 100:.2f}%")
# print(f"Z-statistic: {stat:.4f}")
# print(f"P-value: {pval:.4f}")



Control Group Completion Rate: 59.17%
Test Group Completion Rate: 65.20%


### Hypotese 2


#### Completion Rate with a Cost-Effectiveness Threshold

In [3]:
# calculating Completion Rate with a Cost-Effectiveness Threshold
increase_percentage = ((test_total_completion_rate - control_total_completion_rate) / control_total_completion_rate) * 100

print(f"Control Completion Rate: {control_total_completion_rate * 100:.2f}%")
print(f"Test Completion Rate: {test_total_completion_rate * 100:.2f}%")
print(f"Increase Percentage: {increase_percentage:.2f}%")

Control Completion Rate: 59.17%
Test Completion Rate: 65.20%
Increase Percentage: 10.17%


### Hypotese 3


In [4]:
# Different method

print('Average age of test group:', test_group.mean())
print('Average age of control group:',control_group.mean())
print('Difference of age between two groups:',control_group.mean() - test_group.mean())

Average age of test group: 46.88812666864432
Average age of control group: 47.25968893421724
Difference of age between two groups: 0.37156226557291916


### Metrics


##### Time spent per page in seconds

In [5]:
pivot_table

,confirm,start,step_1,step_2,step_3
Control,18.156915,50.268099,44.247232,86.435402,122.203019
Test,38.794996,50.424246,55.574640,84.843619,120.155537


##### ERROR RATES

In [6]:
error_rates = df_webfullfilter.groupby('experiment')['error'].mean().reset_index()
error_rates.columns = ['experiment', 'error_rate']
error_rates['error_rate'] = error_rates['error_rate'].apply(lambda a : a * 100)


print(f"Erro rates - Control: {error_rates.loc[0,'error_rate']:.2f} %")
print(f"Erro rates - Test: {error_rates.loc[1,'error_rate']:.2f} %")


Erro rates - Control: 6.91 %
Erro rates - Test: 9.51 %


### Hypothesis Testing

In [7]:
### Hypothesis 1 & 2
total_visitors_per_group = df_webfullfilter.groupby('experiment')['visitor_id'].nunique()
confirm_visitors_per_group = df_webfullfilter[df_webfullfilter['process_step'] == 'confirm'].groupby('experiment')['visitor_id'].nunique()
confirmation_rate_per_group = confirm_visitors_per_group / total_visitors_per_group
test_confirmation_rate = confirmation_rate_per_group['Test']
control_confirmation_rate = confirmation_rate_per_group['Control']
n_test = total_visitors_per_group['Test']
n_control = total_visitors_per_group['Control']
confirm_test = confirm_visitors_per_group['Test']
confirm_control = confirm_visitors_per_group['Control']
counts = np.array([confirm_test, confirm_control])
nobs = np.array([n_test, n_control])


#  test z 
stat, p_v = proportions_ztest(counts, nobs, alternative='two-sided')
print(f'Z-statistic: {stat}')
print(f'P-value: {p_v}')


Z-statistic: 14.663372446786198
P-value: 1.106383586182923e-48


In [8]:
# Hypothesis statement

if p_value < alpha:
    result = f"We reject the null hypothesis. There is a significant difference in the average age of clients between the new process and the old process (p-value = {p_value:.5f})."
else:
    result = f"We cannot reject the null hypothesis. There is no significant difference in the average age of clients between the new process and the old process (p-value = {p_value:.5f})."

# Print the results
print(result)
print("\nT-test results")
print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

We reject the null hypothesis. There is a significant difference in the average age of clients between the new process and the old process (p-value = 0.00730).

T-test results
t-statistic: -2.68272662596058
p-value: 0.0073048427495870935
